In [61]:
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import plotly.offline as pyo
import plotly.graph_objs as go


In [4]:
df = pd.read_csv('G:\マイドライブ\TSoM\Data Design\Sales Transaction v.4a.csv\Sales Transaction v.4a.csv')
df.head()

<>:1: SyntaxWarning:

invalid escape sequence '\T'

<>:1: SyntaxWarning:

invalid escape sequence '\T'

C:\Users\mihof\AppData\Local\Temp\ipykernel_16972\4268424036.py:1: SyntaxWarning:

invalid escape sequence '\T'



,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


In [5]:
df.info()
#Found that the data contains 55 Null Value in CostomerNo.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


In [6]:
print(df[df['CustomerNo'].isna()].head(55))
#The observations with Null CustomerNo are mostly cancellation transactions.
#To calculate the accurate sales amount, I don't drop them.

       TransactionNo        Date ProductNo  \
6511         C581406   12/8/2019    46000M   
6512         C581406   12/8/2019    46000S   
90098        C575153   11/8/2019     22947   
102671       C574288   11/3/2019     22178   
117263       C573180  10/28/2019     23048   
163160       C569495   10/4/2019     21843   
190598       C567518   9/20/2019     22846   
192284       C567518   9/20/2019     21871   
242406       C563015   8/11/2019    46000M   
242407       C563015   8/11/2019    46000S   
246334       C562617    8/8/2019     23243   
254250       C561966    8/1/2019     22371   
254254       C562030    8/1/2019    84595E   
260633       C561239   7/26/2019     22427   
260634       C561239   7/26/2019     22426   
260635       C561239   7/26/2019     84755   
276396       C559949   7/14/2019     22699   
285494       C559296    7/7/2019     23001   
285495       C559296    7/7/2019     22616   
285496       C559296    7/7/2019     23298   
297259       C558364   6/28/2019  

In [7]:
#Convert ProductNo and CustomerNo into string
df[['ProductNo','CustomerNo']] = df[['ProductNo','CustomerNo']].astype('str')
df['CustomerNo'] = df['CustomerNo'].apply(lambda x: x.split('.')[0])

#Convert Date into 
df['Date'] = pd.to_datetime(df['Date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   TransactionNo  536350 non-null  object        
 1   Date           536350 non-null  datetime64[ns]
 2   ProductNo      536350 non-null  object        
 3   ProductName    536350 non-null  object        
 4   Price          536350 non-null  float64       
 5   Quantity       536350 non-null  int64         
 6   CustomerNo     536350 non-null  object        
 7   Country        536350 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 32.7+ MB


In [8]:
#Create [Sales] column(price*quantity)
df['Sales'] = df['Price'] * df['Quantity']

In [9]:
#How was the sales trend over the year?
#Data Preparation
df['Month'] = df['Date'].dt.to_period('M')
df_monthly_sales = df.groupby('Month')['Sales'].sum().reset_index()
df_monthly_customers = df.groupby('Month')['CustomerNo'].nunique().reset_index()

df_monthly = pd.merge(df_monthly_sales, df_monthly_customers, on = 'Month')


In [10]:
df_monthly = df_monthly[df_monthly['Month'] != '2019-12'] #Illiminate 2019-12 since that is imcomplete.
df_monthly = df_monthly.set_index('Month')
df_monthly.index = df_monthly.index.to_timestamp()

In [70]:
fig= make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly["Sales"], name="Total Sales", line=dict(color = 'sandybrown')))
fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly["CustomerNo"], name="Active Customers", line=dict(color = 'yellowgreen')),secondary_y=True)
fig.update_traces(hovertemplate='%{y}<br>%{x}')

fig.update_xaxes(showgrid=False, title_text = None)
fig.update_yaxes(title="Total Sales", showgrid=False, range=[0, df_monthly["Sales"].max()*1.05])
fig.update_yaxes(title="Active Customers", showgrid=False, secondary_y=True, range=[0, df_monthly["CustomerNo"].max()*1.05])
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(margin = dict(t=50, b=25),title_text="Monthly Sales and Active Customers Trends", width = 800)

fig.show()
fig.to_html(include_plotlyjs='cdn',full_html=False).encode().decode('unicode-escape')

'<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.30.0.min.js"></script>                <div id="c3567015-ba86-4a5f-9294-63610d023ccf" class="plotly-graph-div" style="height:100%; width:800px;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("c3567015-ba86-4a5f-9294-63610d023ccf")) {                    Plotly.newPlot(                        "c3567015-ba86-4a5f-9294-63610d023ccf",                        [{"line":{"color":"sandybrown"},"name":"Total Sales","x":["2018-12-01T00:00:00","2019-01-01T00:00:00","2019-02-01T00:00:00","2019-03-01T00:00:00","2019-04-01T00:00:00","2019-05-01T00:00:00","2019-06-01T00:00:00","2019-07-01T00:00:00","2019-08-01T00:00:00","2019-09-01T00:00:00","2019-10-01T00:00:00","2019-11-01T00:0

In [12]:
df_product = df.groupby('ProductName')['Sales'].sum().reset_index()
df_product = df_product.sort_values(by = 'Sales', ascending=False).head(15)
df_product

,ProductName,Sales
2489,Popcorn Holder,583286.51
3685,World War 2 Gliders Asstd Designs,557170.43
2210,Paper Craft Little Birdie,501359.05
828,Cream Hanging Heart T-Light Holder,451401.70
204,Assorted Colour Bird Ornament,420743.70
2164,Pack Of 72 Retrospot Cake Cases,388424.61
2549,Rabbit Night Light,327794.21
1679,Jumbo Bag Red Retrospot,290474.52
2661,Regency Cakestand 3 Tier,289288.60
1945,Mini Paint Set Vintage,284870.92


In [69]:
fig = px.bar(df_product, x='Sales', y='ProductName', orientation='h', title="Top Sales Products",
             template="plotly_white", color='Sales', color_continuous_scale='Bluyl')
fig.update_yaxes(categoryorder='total ascending', title_text = None)
fig.update_xaxes(title_text = None)
fig.update_layout(yaxis=dict(tickfont=dict(size=8)))
fig.update_traces(hovertemplate='Product: %{y}<br>Sales: %{x}')
fig.update_layout(margin = dict(t=50, b=25), height=500, width=700, showlegend=False)
fig.show()

fig.to_html(include_plotlyjs='cdn',full_html=False).encode().decode('unicode-escape')

'<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.30.0.min.js"></script>                <div id="85c28cdc-3ea5-47e6-8fea-a9ee52bd8892" class="plotly-graph-div" style="height:500px; width:700px;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("85c28cdc-3ea5-47e6-8fea-a9ee52bd8892")) {                    Plotly.newPlot(                        "85c28cdc-3ea5-47e6-8fea-a9ee52bd8892",                        [{"alignmentgroup":"True","hovertemplate":"Product: %{y}<br>Sales: %{x}","legendgroup":"","marker":{"color":[583286.51,557170.43,501359.05000000005,451401.7,420743.7,388424.61,327794.21,290474.52,289288.6,284870.92,282719.15,269111.27999999997,268681.74,263537.59,253367.32],"coloraxis":"coloraxis","pattern":{"shap

In [14]:
df_country = df.groupby('Country')['Sales'].sum().reset_index()
df_country['Sales Ratio'] = df_country['Sales']/sum(df_country['Sales'])*100

In [63]:
country_color_map = {
    'United Kingdom': 'blue',
    'Norway': 'red',
    'Belgium': 'green',
    'Germany': 'orange',
    'France': 'purple',
    'Austria': 'yellow',
    'Netherlands': 'cyan',
    'EIRE': 'magenta',
    'USA': 'lime',
    'Channel Islands': 'pink',
    'Iceland': 'teal',
    'Portugal': 'brown',
    'Spain': 'navy',
    'Finland': 'olive',
    'Italy': 'maroon',
    'Greece': 'gold',
    'Japan': 'indigo',
    'Sweden': 'silver',
    'Denmark': 'tan',
    'Cyprus': 'azure',
    'Malta': 'violet',
    'Switzerland': 'coral',
    'Australia': 'khaki',
    'Czech Republic': 'orchid',
    'Poland': 'skyblue',
    'Hong Kong': 'plum',
    'Singapore': 'lavender',
    'RSA': 'salmon',
    'Israel': 'crimson',
    'Unspecified': 'lightgreen',
    'United Arab Emirates': 'lightblue',
    'Canada': 'lightcoral',
    'European Community': 'lightpink',
    'Bahrain': 'lightskyblue',
    'Brazil': 'lightsteelblue',
    'Saudi Arabia': 'lightseagreen',
    'Lebanon': 'lightgrey',
    'Lithuania': 'lightyellow'
}

def round_sales_ratio(value):
    return round(value, 2)
df_country['Sales Ratio Rounded'] = df_country['Sales Ratio'].apply(round_sales_ratio)

def add_percentage(value):
    return f"{value}%"
df_country['Sales Ratio Rounded'] = df_country['Sales Ratio Rounded'].apply(add_percentage)

fig1 = px.treemap(df_country, path=['Country'], 
                 values='Sales Ratio', color='Country', color_discrete_map= country_color_map,
                 title='Sales Ratio by Country')
fig1.update_layout(margin = dict(t=50, l=25, r=25, b=25), height=600, width=800)
fig1.update_traces(text=df_country['Sales Ratio Rounded'].astype(str),
                  hoverinfo='label+text+value') 
fig1.show()
fig.to_html(include_plotlyjs='cdn',full_html=False).encode().decode('unicode-escape')

'<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.30.0.min.js"></script>                <div id="e6fcf254-cf9c-4d11-ab69-a3417f242b91" class="plotly-graph-div" style="height:600px; width:800px;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("e6fcf254-cf9c-4d11-ab69-a3417f242b91")) {                    Plotly.newPlot(                        "e6fcf254-cf9c-4d11-ab69-a3417f242b91",                        [{"branchvalues":"total","customdata":[["Australia"],["Austria"],["Bahrain"],["Belgium"],["Brazil"],["Canada"],["Channel Islands"],["Cyprus"],["Czech Republic"],["Denmark"],["EIRE"],["European Community"],["Finland"],["France"],["Germany"],["Greece"],["Hong Kong"],["Iceland"],["Israel"],["Italy"],["Japan"],["Leban

In [25]:
df_customer_total = df.groupby(['Country'])['CustomerNo'].nunique().reset_index()
df_country_per_customer = pd.merge(df_country, df_customer_total, on = 'Country')
df_country_per_customer['Price per Customer'] = df_country_per_customer['Sales']/df_country_per_customer['CustomerNo']
df_country_per_customer = df_country_per_customer.sort_values(by = 'Price per Customer', ascending=False)
df_country_per_customer

,Country,Sales,Sales Ratio,CustomerNo,Price per Customer
24,Netherlands,2147811.39,3.563057,9,238645.710000
10,EIRE,1660645.07,2.754885,13,127741.928462
0,Australia,988756.35,1.640272,9,109861.816667
30,Singapore,63480.95,0.105310,1,63480.950000
32,Sweden,396042.61,0.657005,7,56577.515714
17,Iceland,38321.90,0.063573,1,38321.900000
20,Japan,283293.47,0.469962,8,35411.683750
25,Norway,187544.79,0.311123,8,23443.098750
33,Switzerland,358423.61,0.594598,21,17067.790952
13,France,1316880.98,2.184606,87,15136.562989


In [73]:
def convert_to_k(value):
    return f"{value/1000:.1f} k"
df_country_per_customer['Price per Customer (k)'] = df_country_per_customer['Price per Customer'].apply(convert_to_k)

fig = px.bar(df_country_per_customer, x='Price per Customer', y='Country', orientation='h', title="Price per Customer by Country",
             template="plotly_white", color='Country', color_discrete_map=country_color_map,
             text='Price per Customer (k)')
fig.update_yaxes(categoryorder='total ascending', title_text = None)
fig.update_xaxes(title_text = None)
fig.update_layout(yaxis=dict(tickfont=dict(size=8)))
fig.update_traces(hovertemplate='Country: %{y}<br>Price per Customer: %{x}', textposition='outside', textfont=dict(color='gray'))
fig.update_layout(margin = dict(t=50, b=25), height=600, width=800, showlegend=False)
fig.show()

fig.to_html(include_plotlyjs='cdn',full_html=False).encode().decode('unicode-escape')

'<div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.30.0.min.js"></script>                <div id="bb70921f-19de-448f-8a06-007a5af042a0" class="plotly-graph-div" style="height:600px; width:800px;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("bb70921f-19de-448f-8a06-007a5af042a0")) {                    Plotly.newPlot(                        "bb70921f-19de-448f-8a06-007a5af042a0",                        [{"alignmentgroup":"True","hovertemplate":"Country: %{y}<br>Price per Customer: %{x}","legendgroup":"Netherlands","marker":{"color":"cyan","pattern":{"shape":""}},"name":"Netherlands","offsetgroup":"Netherlands","orientation":"h","showlegend":true,"text":["238.6 k"],"textposition":"outside","x":[238645.71000000002